In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

print("cancer.keys(): {}".format(cancer.keys()))

print("Shape of cancer data: {}".format(cancer.data.shape))